# Семинар 2
# Основы цифровой обработки изображений
# Курс "Компьютерное зрение"

In [ ]:
import time
import numpy as np
from skimage import io
import plotly.graph_objects as go
import cv2

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import scipy.ndimage as sp

## 1. Представление изображений
## 1.1. 2-D RGB изображения

In [ ]:
image_rgb = io.imread("https://www.castlefineart.com/assets/img/resized/standard/the-starry-nightjpg.jpg")

In [ ]:
image_rgb.shape

In [ ]:
image_rgb

In [ ]:
plt.imshow(image_rgb)
plt.show()

## 1.2. 2-D grayscale изображения

In [ ]:
image_gray = io.imread("https://i.stack.imgur.com/B2DBy.jpg", as_gray=True)

In [ ]:
image_gray.shape

In [ ]:
image_gray

In [ ]:
plt.imshow(image_gray, cmap="gray")
plt.show()

## 1.3. 3-D объекты

In [ ]:
vol = io.imread("https://s3.amazonaws.com/assets.datacamp.com/blog_assets/attention-mri.tif")

In [ ]:
vol.shape

In [ ]:
plt.imshow(vol[:,:,0], cmap="gray")
plt.show()

In [ ]:
volume = vol.T
r, c = volume[0].shape
r, c

In [ ]:
# Define frames
nb_frames = 68

In [ ]:
# Define figure
fig = go.Figure(frames=[go.Frame(data=go.Surface(
    z=(6.7 - k * 0.1) * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67 - k]),
    cmin=0, cmax=200
    ),
    name=str(k) # you need to name the frame for the animation to behave properly
    )
    for k in range(nb_frames)])


# Add data to be displayed before animation starts
fig.add_trace(go.Surface(
    z=6.7 * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67]),
    colorscale='Gray',
    cmin=0, cmax=200,
    colorbar=dict(thickness=20, ticklen=4)
    ))


def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }


sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

# Layout
fig.update_layout(
         title='Slices in volumetric data',
         width=600,
         height=600,
         scene=dict(
                    zaxis=dict(range=[-0.1, 6.8], autorange=False),
                    aspectratio=dict(x=1, y=1, z=1),
                    ),
         updatemenus = [
            {
                "buttons": [
                    {
                        "args": [None, frame_args(50)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
         ],
         sliders=sliders
)

fig.show()

## 2. Виды разрешения изображения
## 2.1. Пространственное разрешение

In [ ]:
im_full = cv2.imread('clock.jpg', cv2.IMREAD_GRAYSCALE)

im_resizing = [cv2.resize(im_full, dsize=(int(im_full.shape[0]/i), 
                                          int(im_full.shape[1]/i))) for i in range(2,15,6)]

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(20,15))
axs[0].imshow(im_full, cmap="gray")
axs[0].title.set_text(f'Initial image: {im_full.shape}')

for i, im in enumerate(im_resizing):
    axs[i+1].imshow(im, cmap="gray")
    axs[i+1].title.set_text(f'Decreased image: {im.shape}')
    
plt.plot()

## 2.2. Интерполяция

In [ ]:
list_interpol = [cv2.INTER_NEAREST, cv2.INTER_LINEAR, cv2.INTER_LANCZOS4]
resize_factor = 10
im_resizing_interpol = [cv2.resize(im_full, dsize=(int(im_full.shape[0]/resize_factor), 
                                                   int(im_full.shape[1]/resize_factor)), 
                                   interpolation= interpol_type) for interpol_type in list_interpol]

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(20,15))
axs[0].imshow(im_full, cmap="gray")
axs[0].title.set_text(f'Initial image: {im_full.shape}')

for i, im in enumerate(im_resizing_interpol):
    axs[i+1].imshow(im, cmap="gray")
    axs[i+1].title.set_text(f'Decreased image: {im.shape}')
    
plt.plot()

## 3. Основные математические операции
## 3.1. Произведение

In [ ]:
im_teeth = cv2.imread('teeth.png', cv2.IMREAD_GRAYSCALE)
plt.imshow(im_teeth, cmap="gray")
plt.show()

In [ ]:
im_mask = cv2.imread('mask_for_teeth.png', cv2.IMREAD_GRAYSCALE)
_, im_mask_bin = cv2.threshold(im_mask,127,255,cv2.THRESH_BINARY)
im_mask_bin = cv2.resize(im_mask_bin, dsize=(im_teeth.shape[1],im_teeth.shape[0])) / 255 
plt.imshow(im_mask, cmap="gray")
plt.show()

In [ ]:
im_mask_bin

In [ ]:
im_teeth

In [ ]:
im_result = im_teeth * im_mask_bin
plt.imshow(im_result, cmap="gray")
plt.show()

## 3.2. Суммирование
Усреднение изображений позволяет избавиться от шума.  
Создадим большое количество зашумленных изображений

In [ ]:
im_clock = im_resizing[0]

In [ ]:
mu = 127
sigma = 100
NoiseG = np.random.normal(mu, sigma, (im_clock.shape[0], im_clock.shape[1]))
plt.imshow(NoiseG, cmap='gray')
plt.title('Gaussian noise')
plt.show()

In [ ]:
im_clock_noised = (im_clock + NoiseG)/2
plt.imshow(im_clock_noised, cmap='gray')
plt.show()

In [ ]:
num_images = 1000
noisy_images = [(im_clock + np.random.normal(mu, sigma, (im_clock.shape[0], im_clock.shape[1])) / 2)
               for i in range(num_images)]

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(20,15))
axs[0].imshow(im_clock, cmap="gray")
axs[0].title.set_text(f'Initial image')

for i, im in enumerate(noisy_images[:3]):
    axs[i+1].imshow(im, cmap="gray")
    axs[i+1].title.set_text(f'Noisy image')
    
plt.plot()

In [ ]:
num_noised_images = [100, 500, 1000]
fig, axs = plt.subplots(1, 4, figsize=(20,15))
axs[0].imshow(im_clock, cmap="gray")
axs[0].title.set_text(f'Initial image')

for i, num in enumerate(num_noised_images):
    axs[i+1].imshow(sum(noisy_images[:num])/num, cmap="gray")
    axs[i+1].title.set_text(f'Averaged {num} images')
    
plt.plot()

## 3.3. Разность

In [ ]:
im_nocontr = cv2.imread('no_contrast.png', cv2.IMREAD_GRAYSCALE)
im_contr = cv2.imread('contrast.png', cv2.IMREAD_GRAYSCALE)

fig, axs = plt.subplots(1, 2, figsize=(15,15))
axs[0].imshow(im_nocontr, cmap="gray")
axs[0].title.set_text(f'No contrast')
axs[1].imshow(im_contr, cmap="gray")
axs[1].title.set_text(f'With contrast')

In [ ]:
im_result = (im_nocontr/2 - im_contr/2)
plt.imshow(im_result, cmap='gray')
plt.show()

## 3.4. Аффинные преобразования
OpenCV предлагает два подходна для геометрических трансформаций изображений: 
- афинных - cv.warpAffine (принимает в качестве аргумента матрицу преобразования 2х3)
- преобразований в пространственной перспективе - cv.warpPerspective (принимает матрицу 3х3)  
  
Преобразование смещения:
$$ M = \begin{bmatrix}
1 & 0 & t_{x}\\
0 & 1 & t_{y}\\
\end{bmatrix} $$

In [ ]:
img = cv2.imread('line_pic.png', cv2.IMREAD_GRAYSCALE)
rows,cols = img.shape

M = np.float32([[1, 0, 100], [0, 1, 50]])
dst = cv2.warpAffine(img, M, (cols, rows))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10,10))
axs[0].imshow(img, cmap="gray")
axs[0].title.set_text(f'Initial')
axs[1].imshow(dst, cmap="gray")
axs[1].title.set_text(f'Transformed image')

При аффинном преобразовании все параллельные линии в исходном изображении по-прежнему будут параллельны в выходном изображении. Чтобы найти матрицу преобразования, нам нужны три точки из входного изображения и их соответствующие местоположения в выходном изображении. Затем cv.getAffineTransform создаст матрицу 2x3, которая будет передана в cv.warpAffine.

In [ ]:
img = cv2.imread('line_pic.png')
rows,cols,ch = img.shape

pts1 = np.float32([[50,50],[200,50],[50,200]])
pts2 = np.float32([[10,100],[200,50],[100,250]])

M = cv2.getAffineTransform(pts1,pts2)
dst = cv2.warpAffine(img,M,(cols,rows))

plt.subplot(121),plt.imshow(img),plt.title('Input')
plt.subplot(122),plt.imshow(dst),plt.title('Output')
plt.show()

Для преобразования в перспективе требуется матрица преобразования 3x3. Прямые линии останутся прямыми даже после трансформации. Чтобы найти эту матрицу преобразования, нам нужны 4 точки на входном изображении и соответствующие точки на выходном изображении. Среди этих 4 точек 3 из них не должны лежать на одной прямой. Тогда матрицу преобразования можно найти с помощью функции cv.getPerspectiveTransform. Затем примените cv.warpPerspective с этой матрицей преобразования 3x3.

In [ ]:
img = cv2.imread('sudoku.png')

rows,cols,ch = img.shape

pts1 = np.float32([[115,130],[746,106],[60,775],[788,784]])
pts2 = np.float32([[0,0],[300,0],[0,300],[300,300]])

M = cv2.getPerspectiveTransform(pts1,pts2)
dst = cv2.warpPerspective(img,M,(300,300))

plt.subplot(121),plt.imshow(img),plt.title('Input')
plt.subplot(122),plt.imshow(dst),plt.title('Output')
plt.show()

## 3.5. Статистические метрики
Оценка дисперсии изображений с целью оценки сфокусированности

In [ ]:
micro_images = []
variance_vals = []

fig, axs = plt.subplots(1, 5, figsize=(20,15))

for i in range(1, 6):
    img = cv2.imread(f'{i}.jpg', cv2.IMREAD_GRAYSCALE)
    var_val = np.var(img)
    
    axs[i-1].imshow(img, cmap="gray")
    axs[i-1].title.set_text(f'Variance value: {var_val :.2f}')
    
plt.plot()

## 4. Пространственная фильтрация
## 4.1. Свертка
<img src="conv1.gif" width="400" height="400">
<img src="conv2.gif" width="400" height="400">

Как и в случае одномерных сигналов, изображения также могут быть отфильтрованы с помощью различных фильтров нижних частот (LPF), фильтров верхних частот (HPF) и т. д. LPF помогает удалять шум, размывать изображения, фильтры HPF помогают находить края в изображений.  

OpenCV предоставляет функцию cv.filter2D () для свертки ядра с изображением. В качестве примера мы попробуем использовать усредняющий фильтр на изображении. Ядро усредняющего фильтра 5x5 будет выглядеть следующим образом:
$$ K = 1/25\begin{bmatrix}
1 & 1 & 1 & 1 & 1\\
1 & 1 & 1 & 1 & 1\\
1 & 1 & 1 & 1 & 1\\
1 & 1 & 1 & 1 & 1\\
1 & 1 & 1 & 1 & 1\\
\end{bmatrix} $$

In [ ]:
kernel = np.ones((5,5), np.float32)/25
dst = cv2.filter2D(image_gray,-1,kernel)

plt.figure(figsize=(10, 6))
plt.subplot(121),plt.imshow(image_gray, cmap="gray"),plt.title('Original')
plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(dst, cmap="gray"),plt.title('Averaging')
plt.xticks([]), plt.yticks([])
plt.show()

Реализуем функцию для расчета 2-D свертки с ядром 3х3:

In [ ]:
def convolve2d(image, kernel):
    # convolution output
    output = np.zeros_like(image)

    # Add zero padding to the input image
    image_padded = np.zeros((image.shape[0] + 2, image.shape[1] + 2))
    image_padded[1:-1, 1:-1] = image

    # Loop over every pixel of the image
    for x in range(image.shape[1]):
        for y in range(image.shape[0]):
            # element-wise multiplication of the kernel and the image
            output[y, x]=(kernel * image_padded[y: y+3, x: x+3]).sum()

    return output

In [ ]:
kernel = np.ones((3,3), np.float32)/9
dst_manual = convolve2d(image_gray,kernel)

plt.figure(figsize=(15, 10))
plt.subplot(131),plt.imshow(image_gray, cmap="gray"),plt.title('Original')
plt.xticks([]), plt.yticks([])
plt.subplot(132),plt.imshow(dst, cmap="gray"),plt.title('Averaging builtin')
plt.xticks([]), plt.yticks([])
plt.subplot(133),plt.imshow(dst, cmap="gray"),plt.title('Averaging manual')
plt.xticks([]), plt.yticks([])
plt.show()

## 4.2. Сглаживание изображений
Применим нелинейный метод фильтрации, выполняющийся, как и в предыдущем случае, в скользящем окне. 
Применим встроенную функцию cv.medianBlur(), которая определяет медианное значение всех пикселей под областью ядра, а центральный элемент заменяется этим медианным значением. Это очень эффективно против шума по типу "солью и перцем" на изображении. Это эффективно снижает шум. Размер ядра должен быть положительным нечетным целым числом.

In [ ]:
image_noise = io.imread("img_noise.png")
image_median = cv2.medianBlur(image_noise, 9)

plt.figure(figsize=(10, 6))
plt.subplot(121),plt.imshow(image_noise),plt.title('Noised')
plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(image_median),plt.title('After median filtering')
plt.xticks([]), plt.yticks([])
plt.show()

Реализуем вручную медианную фильтрацию:

In [ ]:
image_noise = io.imread("img_noise.png", as_gray=True)
image_noise = cv2.resize(image_noise, dsize=(300,300))

w = 5

image_median_man = np.empty([image_noise.shape[0], image_noise.shape[1]])
for i in range(0, image_noise.shape[0]):
    for j in range(0, image_noise.shape[1]):
        image_median_man[i,j] = np.median(image_noise[(i-int(w/2)):(i+int(w/2)), (j-int(w/2)):(j+int(w/2))])
        
plt.imshow(image_median_man, cmap='gray')
plt.axis('off')
plt.title('Manual implementation of median filtering')
plt.show()

## 4.3. Вычисление градиентов
Выделим границы с помощью матрицы Лапласа. Такой подход позволяет выделить за один проход границы различной пространственной ориентации:
$$ kernel = \begin{bmatrix}
0 & 1 & 0\\
1 & -4 & 1\\
0 & 1 & 0\\
\end{bmatrix} $$

In [ ]:
image_sudoku = cv2.imread("cross.png", 0)

laplacian = cv2.Laplacian(image_sudoku, cv2.CV_64F)
sobelx = cv2.Sobel(image_sudoku, cv2.CV_64F, 1, 0, ksize=5)
sobely = cv2.Sobel(image_sudoku, cv2.CV_64F, 0, 1, ksize=5)

plt.figure(figsize=(10, 10))
plt.subplot(2,2,1), plt.imshow(image_sudoku, cmap = 'gray')
plt.title('Original'), plt.xticks([]), plt.yticks([])

plt.subplot(2,2,2),plt.imshow(laplacian, cmap = 'gray')
plt.title('Laplacian'), plt.xticks([]), plt.yticks([])

plt.subplot(2,2,3),plt.imshow(sobelx, cmap = 'gray')
plt.title('Sobel X'), plt.xticks([]), plt.yticks([])

plt.subplot(2,2,4),plt.imshow(sobely, cmap = 'gray')
plt.title('Sobel Y'), plt.xticks([]), plt.yticks([])
plt.show()

## 4.4. Поиск шаблона на изображении
Сопоставление шаблонов - это метод поиска и определения местоположения изображения шаблона на исходном изображении. OpenCV предлагает функцию cv.matchTemplate() для этой цели. Идея заключается в прохождении шаблона по входному изображению (как при двумерной свертке) и сравнении шаблона с фрагментом входного изображения. В OpenCV реализовано несколько методов сравнения. Функция возвращает изображение в градациях серого, где каждый пиксель обозначает, насколько данная область соответствует шаблону.

In [ ]:
img = cv2.imread('messi.jpg',0)
img2 = img.copy()
template = cv2.imread('template.jpg',0)

plt.imshow(template,cmap = 'gray')
plt.show()

In [ ]:
w, h = template.shape[::-1]

meth = 'cv2.TM_CCOEFF'

img = img2.copy()
method = eval(meth)

# Apply template Matching
res = cv2.matchTemplate(img,template,method)
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

top_left = max_loc
bottom_right = (top_left[0] + w, top_left[1] + h)
cv2.rectangle(img,top_left, bottom_right, 255, 2)

plt.figure(figsize=(10, 10))
plt.subplot(121),plt.imshow(res,cmap = 'gray')
plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(img,cmap = 'gray')
plt.title('Detected Point'), plt.xticks([]), plt.yticks([])

plt.show()

## 5. Яркостные преобразования
## 5.1. Основы
<img src="Graph.png" width="400" height="400">
<img src="Graph1.png" width="400" height="400">

In [ ]:
img_breast = cv2.imread('breast.png',0)
img_breast

In [ ]:
# Broadcasting
img_breast_negative = 255 - img_breast
img_breast_negative

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(121),plt.imshow(img_breast,cmap = 'gray')
plt.title('Initial image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(img_breast_negative,cmap = 'gray')
plt.title('Negative image'), plt.xticks([]), plt.yticks([])
plt.show()

## 5.2. Гистограмма изображения

In [ ]:
img_scene = cv2.imread('scene.png',0)
hist_full = cv2.calcHist([img_scene],[0],None,[256],[0,256])

plt.figure(figsize=(15, 5))
plt.subplot(121), plt.imshow(img_scene, 'gray')
plt.subplot(122), plt.plot(hist_full),plt.xlim([0,256])
plt.show()

## 5.3. Эквализация гистограммы
<img src="hist_eq.png" width="400" height="400">
  
Выполним глобальную эквализацию гистограммы

In [ ]:
img_eq_glob = cv2.equalizeHist(img_scene)
hist_eq_scene = cv2.calcHist([img_eq_glob],[0],None,[256],[0,256])

plt.figure(figsize=(15, 5))
plt.subplot(121), plt.imshow(img_eq_glob, 'gray')
plt.subplot(122), plt.plot(hist_eq_scene),plt.xlim([0,256])
plt.show()

Выполним адаптивную локальную эквализацию гистограммы: CLAHE (Contrast Limited Adaptive Histogram Equalization).   
В результате глобальной эквализации мы потеряли часть информации из-за чрезмерной яркости (лицо). Для решения этой проблемы используется адаптивное выравнивание гистограммы. В этом случае изображение разделяется на небольшие блоки, называемые «плитками» (размер плитки по умолчанию в OpenCV равен 8x8). Затем для каждого из этих блоков выравнивается гистограмма.

In [ ]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
img_clahe = clahe.apply(img_scene)
hist_eq_clahe = cv2.calcHist([img_clahe],[0],None,[256],[0,256])

plt.figure(figsize=(15, 5))
plt.subplot(121), plt.imshow(img_clahe, 'gray')
plt.subplot(122), plt.plot(hist_eq_clahe),plt.xlim([0,256])
plt.show()

## 5.4. Извлечение объектов по гистограмме
Извлечение объектов по гистограмме (Histogram backprojection) было предложено Майклом Дж. Суэйном, Даной Х. Баллард в их статье «Indexing via color histograms».

Данный подход используется для сегментации изображения или поиска интересующих объектов на изображении. В результате применения такого метода создается изображение того же размера (но с одним каналом), что и у входного изображения, где каждый пиксель соответствует вероятности того, что он принадлежит искомому объекту.   

OpenCV предоставляет встроенную функцию cv.calcBackProject(). Ее параметры почти такие же, как у функции cv.calcHist(). Один из его параметров - гистограмма искомого объекта. Кроме того, гистограмма объекта должна быть нормализована перед применением функции backproject. Функция возвращает вероятностное изображение, которое далее сворачивается с исходным изображением.

In [ ]:
roi = cv2.imread('temp_hist.jpg') 
hsv = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)

target = cv2.imread('messi.jpg')
hsvt = cv2.cvtColor(target,cv2.COLOR_BGR2HSV)

# calculating object histogram
roihist = cv2.calcHist([hsv],[0, 1], None, [180, 256], [0, 180, 0, 256] )

# normalize histogram and apply backprojection
cv2.normalize(roihist,roihist,0,255,cv2.NORM_MINMAX)
dst = cv2.calcBackProject([hsvt],[0,1],roihist,[0,180,0,256],1)

# Now convolute with circular disc
disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
cv2.filter2D(dst,-1,disc,dst)

# threshold and binary AND
ret,thresh = cv2.threshold(dst,50,255,0)
thresh = cv2.merge((thresh,thresh,thresh))
res = cv2.bitwise_and(target,thresh)

# Merge images
res = cv2.cvtColor(np.vstack((target,thresh,res)), cv2.COLOR_BGR2RGB) 

plt.figure(figsize=(15, 15))
plt.imshow(res)
plt.show()

## Задание
1. С помощью OpenCV и веб-камеры получить изображение документа на листе A4
2. Перевести изображение в серошкальный формат
3. Определить координаты угловых точек документа с помощью интерактивный утилиты matplotlib (см. семинар 1)
4. Выполнить преобразование перспективы так, чтобы плоскость документа была выровнена (эффект сканирования)
5. Провести адаптивную эквализацию гистограммы
6. Выделить на изображении все границы
7. Получить маску по границам
8. Применить маску к изображению, полученному на этапе 5.